# Logistic Features for TDE Classification

This notebook calls a compact logistic feature extractor implemented in `model.py` to train on the provided lightcurves and produce the competition submission.

In [ ]:
import runpy
runpy.run_path('model.py')